In [1]:
import pandas as pd
import numpy as np
import wfdb
import ast

In [2]:
import os

In [3]:
def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(os.path.join(path,f)) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(os.path.join(path,f)) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

In [4]:
path='/kaggle/input/ecg-data/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3/'
sampling_rate=100

# load and convert annotation data
Y = pd.read_csv(os.path.join(path,'ptbxl_database.csv'), index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))


In [5]:
Y.scp_codes 

ecg_id
1                 {'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}
2                             {'NORM': 80.0, 'SBRAD': 0.0}
3                               {'NORM': 100.0, 'SR': 0.0}
4                               {'NORM': 100.0, 'SR': 0.0}
5                               {'NORM': 100.0, 'SR': 0.0}
                               ...                        
21833    {'NDT': 100.0, 'PVC': 100.0, 'VCLVH': 0.0, 'ST...
21834             {'NORM': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
21835                           {'ISCAS': 50.0, 'SR': 0.0}
21836                           {'NORM': 100.0, 'SR': 0.0}
21837                           {'NORM': 100.0, 'SR': 0.0}
Name: scp_codes, Length: 21799, dtype: object

In [6]:
Y.scp_codes.shape

(21799,)

In [7]:
X = load_raw_data(Y, sampling_rate, path)


In [8]:
agg_df = pd.read_csv(os.path.join(path,'scp_statements.csv'), index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]


In [9]:
def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

In [10]:
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

In [11]:
test_fold = 10

X_train = X[np.where(Y.strat_fold != test_fold)]
y_train = Y[(Y.strat_fold != test_fold)].diagnostic_superclass

X_test = X[np.where(Y.strat_fold == test_fold)]
y_test = Y[Y.strat_fold == test_fold].diagnostic_superclass

In [12]:
X_train.shape

(19601, 1000, 12)

In [13]:
y_train.head

<bound method NDFrame.head of ecg_id
1        [NORM]
2        [NORM]
3        [NORM]
4        [NORM]
5        [NORM]
          ...  
21833    [STTC]
21834    [NORM]
21835    [STTC]
21836    [NORM]
21837    [NORM]
Name: diagnostic_superclass, Length: 19601, dtype: object>

In [14]:
y_train.info()

<class 'pandas.core.series.Series'>
Index: 19601 entries, 1 to 21837
Series name: diagnostic_superclass
Non-Null Count  Dtype 
--------------  ----- 
19601 non-null  object
dtypes: object(1)
memory usage: 306.3+ KB


In [15]:
np.unique(y_train).shape[0]


22

In [16]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
y_train_encoded = pd.DataFrame(mlb.fit_transform(y_train), columns=mlb.classes_)

In [17]:
y_train_encoded.shape

(19601, 5)

In [18]:
y_train_encoded.head()

,CD,HYP,MI,NORM,STTC
0,0,0,0,1,0
1,0,0,0,1,0
2,0,0,0,1,0
3,0,0,0,1,0
4,0,0,0,1,0


In [19]:
type(y_train_encoded)

pandas.core.frame.DataFrame

In [20]:
y_train_encoded['final_output']=y_train_encoded.apply(lambda x:''.join(x.astype(str)),axis=1)

In [21]:
y_train_encoded.head()

,CD,HYP,MI,NORM,STTC,final_output
0,0,0,0,1,0,00010
1,0,0,0,1,0,00010
2,0,0,0,1,0,00010
3,0,0,0,1,0,00010
4,0,0,0,1,0,00010


In [22]:
y_train_encoded['final_output'].nunique()

22

In [23]:
y_train_encoded['final_output'].unique()

array(['00010', '00100', '00000', '00001', '01000', '10000', '00101',
       '11000', '10100', '10001', '01100', '01101', '01001', '10101',
       '10010', '11101', '11001', '00011', '11100', '10011', '01010',
       '11110'], dtype=object)

In [24]:
def adjust_classes(y_train):
    var=y_train
    if var == '00010':
        var=0
    elif var == '00100':
        var =1
    elif var== '00000':
        var=2
    elif var=='00001':
        var =3
    elif var =='01000':
        var=4
    elif var=='10000':
        var =5
    elif var =='00101':
        var =6
    elif var =='11000':
        var=7
    elif var =='10100':
        var=8
    elif var=='10001':
        var =9
    elif var =='01100':
        var =10
    elif var =='01101':
        var =11
    elif var=='01001':
        var=21
    elif var =='10101':
        var=12
    elif var =='10010':
        var=13
    elif var=='11101':
        var=14
    elif var=='11001':
        var=15
    elif var =='00011':
        var=16
    elif var=='11100':
        var=17
    elif var=='10011':
        var=18
    elif var =='01010':
        var=19
    elif var=='11110':
        var=20
    return var

In [25]:
y_train_encoded['final_output']=y_train_encoded['final_output'].apply(adjust_classes)

In [26]:
y_train_encoded

,CD,HYP,MI,NORM,STTC,final_output
0,0,0,0,1,0,0
1,0,0,0,1,0,0
2,0,0,0,1,0,0
3,0,0,0,1,0,0
4,0,0,0,1,0,0
...,...,...,...,...,...,...
19596,0,0,0,0,1,3
19597,0,0,0,1,0,0
19598,0,0,0,0,1,3
19599,0,0,0,1,0,0


In [27]:
y_train_encoded['final_output'].value_counts()

final_output
0     8157
1     2276
3     2158
5     1524
8     1164
21     708
6      538
4      479
9      433
2      371
13     362
11     320
7      273
12     202
15     186
10     166
14     140
17     112
16      24
18       5
19       2
20       1
Name: count, dtype: int64

In [28]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [29]:
x_train_normalization=X_train.reshape(-1,1000*12)

In [30]:
x_train=scaler.fit_transform(x_train_normalization)
x_train_normalized=x_train.reshape(19601,1000,12)

In [31]:
class_distribution_norm=y_train_encoded['NORM'].value_counts()
class_distribution_norm

NORM
0    11050
1     8551
Name: count, dtype: int64

In [32]:
class_distribution_cd=y_train_encoded['CD'].value_counts()
class_distribution_cd

CD
0    15199
1     4402
Name: count, dtype: int64

In [33]:
class_distribution_hyp=y_train_encoded['HYP'].value_counts()
class_distribution_hyp

HYP
0    17214
1     2387
Name: count, dtype: int64

In [34]:
class_distribution_mi=y_train_encoded['MI'].value_counts()
class_distribution_mi

MI
0    14682
1     4919
Name: count, dtype: int64

In [35]:
class_distribution_sttc=y_train_encoded['STTC'].value_counts()
class_distribution_sttc

STTC
0    14887
1     4714
Name: count, dtype: int64

In [36]:
indices_0 = np.where(y_train_encoded['NORM'] == 0)[0] 
indices_1 = np.where(y_train_encoded['NORM'] == 1)[0]
selected_indices_0=np.random.choice(indices_0, 4000, replace=False)
selected_indices_1=np.random.choice(indices_1, 4000, replace=False)
x_train_norm0=X_train[selected_indices_0]
x_train_norm1=X_train[selected_indices_1]
y_train_norm0=y_train_encoded.iloc[selected_indices_0,3]
y_train_norm1=y_train_encoded.iloc[selected_indices_1,3]
x_train= np.concatenate([x_train_norm0, x_train_norm1], axis=0)
y_train=np.concatenate([y_train_norm0, y_train_norm1], axis=0)

In [37]:
y_train.shape

(8000,)

In [38]:
x_train.shape

(8000, 1000, 12)

In [ ]:
##HYBRID CNN-RNN MODEL

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Dropout, LSTM, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

# Assuming x_train and y_train are already defined with shapes (num_samples, 1000, 12) and (num_samples,)
# Update your x_train shape before fitting the model
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], x_train.shape[2]))  # Ensure shape is (num_samples, 1000, 12)

model = Sequential()

# Convolutional layers
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(1000, 12), kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv1D(128, kernel_size=3, activation='relu', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv1D(256, kernel_size=3, activation='relu', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.4))

model.add(LSTM(100, return_sequences=False, dropout=0.4, recurrent_dropout=0.2))

model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.4))
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.3))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

x_train, val_data, y_train, val_labels = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42
)

history = model.fit(
    x_train, y_train,
    epochs=100, 
    batch_size=32, 
    validation_data=(val_data, val_labels), 
    callbacks=[early_stopping]
)


Epoch 1/100
160/160 ━━━━━━━━━━━━━━━━━━━━ 32s 175ms/step - accuracy: 0.5720 - loss: 1.0575 - val_accuracy: 0.4938 - val_loss: 1.2533
Epoch 2/100
160/160 ━━━━━━━━━━━━━━━━━━━━ 27s 171ms/step - accuracy: 0.7718 - loss: 0.8411 - val_accuracy: 0.5250 - val_loss: 1.1774
Epoch 3/100
160/160 ━━━━━━━━━━━━━━━━━━━━ 27s 170ms/step - accuracy: 0.8061 - loss: 0.7397 - val_accuracy: 0.6977 - val_loss: 0.9731
Epoch 4/100
160/160 ━━━━━━━━━━━━━━━━━━━━ 27s 171ms/step - accuracy: 0.8204 - loss: 0.6970 - val_accuracy: 0.7805 - val_loss: 0.7457
Epoch 5/100
160/160 ━━━━━━━━━━━━━━━━━━━━ 27s 171ms/step - accuracy: 0.8234 - loss: 0.6518 - val_accuracy: 0.8258 - val_loss: 0.6424
Epoch 6/100
160/160 ━━━━━━━━━━━━━━━━━━━━ 27s 171ms/step - accuracy: 0.8423 - loss: 0.5919 - val_accuracy: 0.8602 - val_loss: 0.5296
Epoch 7/100
160/160 ━━━━━━━━━━━━━━━━━━━━ 27s 171ms/step - accuracy: 0.8349 - loss: 0.5776 - val_accuracy: 0.8687 - val_loss: 0.5074
Epoch 8/100
160/160 ━━━━━━━━━━━━━━━━━━━━ 27s 171ms/step - accuracy: 0.8413 -

In [51]:
indices_0 = np.where(y_train_encoded['CD'] == 0)[0] 
indices_1 = np.where(y_train_encoded['CD'] == 1)[0]
selected_indices_0=np.random.choice(indices_0, 4000, replace=False)
selected_indices_1=np.random.choice(indices_1, 4000, replace=False)
x_train_norm0=X_train[selected_indices_0]
x_train_norm1=X_train[selected_indices_1]
y_train_norm0=y_train_encoded.iloc[selected_indices_0,3]
y_train_norm1=y_train_encoded.iloc[selected_indices_1,3]
x_train= np.concatenate([x_train_norm0, x_train_norm1], axis=0)
y_train=np.concatenate([y_train_norm0, y_train_norm1], axis=0)
x_train, val_data, y_train, val_labels = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42
)

# Training the model
history = model.fit(
    x_train, y_train,
    epochs=100, 
    batch_size=32, 
    validation_data=(val_data, val_labels), 
    callbacks=[early_stopping]
)

Epoch 1/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 34s 172ms/step - accuracy: 0.8604 - loss: 0.4103 - val_accuracy: 0.8750 - val_loss: 0.3812
Epoch 2/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 35s 173ms/step - accuracy: 0.8668 - loss: 0.4014 - val_accuracy: 0.8594 - val_loss: 0.3910
Epoch 3/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 34s 170ms/step - accuracy: 0.8573 - loss: 0.3991 - val_accuracy: 0.8712 - val_loss: 0.3772
Epoch 4/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 34s 170ms/step - accuracy: 0.8695 - loss: 0.3826 - val_accuracy: 0.8644 - val_loss: 0.3792
Epoch 5/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 34s 169ms/step - accuracy: 0.8651 - loss: 0.3909 - val_accuracy: 0.8706 - val_loss: 0.3909
Epoch 6/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 41s 169ms/step - accuracy: 0.8602 - loss: 0.3977 - val_accuracy: 0.8725 - val_loss: 0.3904
Epoch 7/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 34s 169ms/step - accuracy: 0.8679 - loss: 0.3859 - val_accuracy: 0.8519 - val_loss: 0.4149
Epoch 8/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 34s 171ms/step - accuracy: 0.8648 -

In [52]:
indices_0 = np.where(y_train_encoded['STTC'] == 0)[0] 
indices_1 = np.where(y_train_encoded['STTC'] == 1)[0]
selected_indices_0=np.random.choice(indices_0, 4000, replace=False)
selected_indices_1=np.random.choice(indices_1, 4000, replace=False)
x_train_norm0=X_train[selected_indices_0]
x_train_norm1=X_train[selected_indices_1]
y_train_norm0=y_train_encoded.iloc[selected_indices_0,3]
y_train_norm1=y_train_encoded.iloc[selected_indices_1,3]
x_train= np.concatenate([x_train_norm0, x_train_norm1], axis=0)
y_train=np.concatenate([y_train_norm0, y_train_norm1], axis=0)
x_train, val_data, y_train, val_labels = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42
)

# Training the model
history = model.fit(
    x_train, y_train,
    epochs=100, 
    batch_size=32, 
    validation_data=(val_data, val_labels), 
    callbacks=[early_stopping]
)

Epoch 1/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 34s 168ms/step - accuracy: 0.8690 - loss: 0.3798 - val_accuracy: 0.8900 - val_loss: 0.3511
Epoch 2/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 34s 170ms/step - accuracy: 0.8779 - loss: 0.3733 - val_accuracy: 0.8863 - val_loss: 0.3560
Epoch 3/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 34s 168ms/step - accuracy: 0.8885 - loss: 0.3528 - val_accuracy: 0.8950 - val_loss: 0.3389
Epoch 4/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 34s 169ms/step - accuracy: 0.8817 - loss: 0.3685 - val_accuracy: 0.9044 - val_loss: 0.3379
Epoch 5/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 34s 170ms/step - accuracy: 0.8874 - loss: 0.3640 - val_accuracy: 0.8844 - val_loss: 0.3666
Epoch 6/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 34s 170ms/step - accuracy: 0.8839 - loss: 0.3570 - val_accuracy: 0.8994 - val_loss: 0.3361
Epoch 7/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 34s 170ms/step - accuracy: 0.8941 - loss: 0.3458 - val_accuracy: 0.8825 - val_loss: 0.3965
Epoch 8/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 34s 171ms/step - accuracy: 0.8829 -

In [53]:
indices_0 = np.where(y_train_encoded['MI'] == 0)[0] 
indices_1 = np.where(y_train_encoded['MI'] == 1)[0]
selected_indices_0=np.random.choice(indices_0, 4000, replace=False)
selected_indices_1=np.random.choice(indices_1, 4000, replace=False)
x_train_norm0=X_train[selected_indices_0]
x_train_norm1=X_train[selected_indices_1]
y_train_norm0=y_train_encoded.iloc[selected_indices_0,3]
y_train_norm1=y_train_encoded.iloc[selected_indices_1,3]
x_train= np.concatenate([x_train_norm0, x_train_norm1], axis=0)
y_train=np.concatenate([y_train_norm0, y_train_norm1], axis=0)
x_train, val_data, y_train, val_labels = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42
)

# Training the model
history = model.fit(
    x_train, y_train,
    epochs=100, 
    batch_size=32, 
    validation_data=(val_data, val_labels), 
    callbacks=[early_stopping]
)

Epoch 1/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 34s 170ms/step - accuracy: 0.8729 - loss: 0.3773 - val_accuracy: 0.8813 - val_loss: 0.3716
Epoch 2/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 34s 169ms/step - accuracy: 0.8706 - loss: 0.3795 - val_accuracy: 0.8619 - val_loss: 0.4292
Epoch 3/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 33s 167ms/step - accuracy: 0.8727 - loss: 0.3665 - val_accuracy: 0.8775 - val_loss: 0.3799
Epoch 4/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 34s 168ms/step - accuracy: 0.8723 - loss: 0.3839 - val_accuracy: 0.8750 - val_loss: 0.3743
Epoch 5/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 33s 167ms/step - accuracy: 0.8795 - loss: 0.3671 - val_accuracy: 0.8819 - val_loss: 0.3744
Epoch 6/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 33s 167ms/step - accuracy: 0.8740 - loss: 0.3767 - val_accuracy: 0.8775 - val_loss: 0.3748
Epoch 7/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 34s 168ms/step - accuracy: 0.8755 - loss: 0.3756 - val_accuracy: 0.8819 - val_loss: 0.3701
Epoch 8/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 33s 166ms/step - accuracy: 0.8869 -

In [54]:
indices_0 = np.where(y_train_encoded['HYP'] == 0)[0] 
indices_1 = np.where(y_train_encoded['HYP'] == 1)[0]
selected_indices_0=np.random.choice(indices_0, 2000, replace=False)
selected_indices_1=np.random.choice(indices_1, 2000, replace=False)
x_train_norm0=X_train[selected_indices_0]
x_train_norm1=X_train[selected_indices_1]
y_train_norm0=y_train_encoded.iloc[selected_indices_0,3]
y_train_norm1=y_train_encoded.iloc[selected_indices_1,3]
x_train= np.concatenate([x_train_norm0, x_train_norm1], axis=0)
y_train=np.concatenate([y_train_norm0, y_train_norm1], axis=0)
x_train, val_data, y_train, val_labels = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42
)

# Training the model
history = model.fit(
    x_train, y_train,
    epochs=100, 
    batch_size=32, 
    validation_data=(val_data, val_labels), 
    callbacks=[early_stopping]
)

Epoch 1/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 17s 167ms/step - accuracy: 0.8905 - loss: 0.3493 - val_accuracy: 0.8550 - val_loss: 0.3697
Epoch 2/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 17s 167ms/step - accuracy: 0.8935 - loss: 0.3481 - val_accuracy: 0.8763 - val_loss: 0.3642
Epoch 3/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 17s 168ms/step - accuracy: 0.8930 - loss: 0.3258 - val_accuracy: 0.8712 - val_loss: 0.3604
Epoch 4/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 17s 168ms/step - accuracy: 0.8957 - loss: 0.3347 - val_accuracy: 0.8788 - val_loss: 0.3606
Epoch 5/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 20s 167ms/step - accuracy: 0.9036 - loss: 0.3140 - val_accuracy: 0.8750 - val_loss: 0.3517
Epoch 6/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 21s 171ms/step - accuracy: 0.8987 - loss: 0.3336 - val_accuracy: 0.8712 - val_loss: 0.3692
Epoch 7/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 17s 167ms/step - accuracy: 0.8865 - loss: 0.3392 - val_accuracy: 0.8562 - val_loss: 0.4217
Epoch 8/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 17s 168ms/step - accuracy: 0.8969 -